In [ ]:
#Q1 List the facilities that do not charge a fee to members
SELECT * 
FROM Facilities
WHERE membercost > 0

In [3]:
#Q2 How many facilities do not charge a fee to members?
SELECT * 
FROM Facilities 
WHERE membercost = 0
#Shows 4 facilities that do not charge a fee to members

In [ ]:
#Q3 Produce a list of facilities that charge a fee less than 20% of the facility's monthly maintenance
#Return the facid, facility name, member cost, and monthly maintenance
SELECT *
FROM Facilities
WHERE membercost < (0.2)*monthlymaintenance

In [ ]:
#Q4 How can you retrieve the details of facilities with ID 1 and 5 without using OR?
SELECT *
FROM Facilities
WHERE facid IN ( 1, 5 ) 

In [ ]:
#Q5 How can you produce a list of facilities, with each labelled as 'cheap' or 'expensive', depending on if their monthly maintenance cost is more than $100? Return the name and monthly maintenance of the facilities in question. 
SELECT name, 
CASE WHEN monthlymaintenance <100 THEN  'cheap' ELSE  'expensive'
END AS cheap_or_expensive
FROM Facilities

In [ ]:
#Q6: You'd like to get the first and last name of the last member(s) who signed up. Do not use the LIMIT clause for your solution. 
SELECT firstname, surname, joindate FROM Members
WHERE joindate = (SELECT MAX(joindate) 
FROM Members)

In [ ]:
#Q7: How can you produce a list of all members who have used a tennis court? Include in your output the name of the court, and the name of the member formatted as a single column. Ensure no duplicate data, and order by the member name. 
SELECT CONCAT( m.surname,  ",",  " ", m.firstname ) AS member, f.name AS name
FROM  `Bookings` b
JOIN  `Facilities` f ON b.facid = f.facid
JOIN  `Members` m ON b.memid = m.memid
WHERE f.name LIKE  '%Tennis Court%'
GROUP BY 1 , 2
ORDER BY member

In [ ]:
#Q8: How can you produce a list of bookings on the day of 2012-09-14 which will cost the member (or guest) more than $30? Remember that guests have different costs to members (the listed costs are per half-hour 'slot'), and the guest user's ID is always 0. Include in your output the name of the facility, the name of the member formatted as a single column, and the cost. Order by descending cost, and do not use any subqueries.
SELECT CONCAT( m.firstname,  '', m.surname ) AS member, f.name AS facility,
(CASE WHEN m.memid =0
THEN f.guestcost * b.slots
ELSE f.membercost * b.slots
END) AS cost
FROM Members AS m
INNER JOIN Bookings AS b ON ( m.memid = b.memid ) 
INNER JOIN Facilities AS f ON ( b.facid = f.facid ) 
WHERE (b.starttime LIKE  '2012-09-14%')
AND ((m.memid =0 AND b.slots * f.guestcost >30)
OR (m.memid >0 AND b.slots * f.membercost >30))
ORDER BY cost DESC 


In [ ]:
#Q9: This time, produce the same result as in Q8, but using a subquery. 
SELECT member, facility, cost
FROM (SELECT 
      CONCAT( m.firstname,  '', m.surname ) AS member, f.name AS facility, 
CASE WHEN m.memid =0
THEN b.slots * f.guestcost
ELSE b.slots * f.membercost
END AS cost
FROM Members AS m
INNER JOIN Bookings AS b ON m.memid = b.memid
INNER JOIN Facilities AS f ON b.facid = f.facid
WHERE b.starttime LIKE  '2012-09-14%') AS bookings
WHERE cost >30
ORDER BY cost DESC

In [ ]:
#Q10: Produce a list of facilities with a total revenue less than 1000. The output of facility name and total revenue, sorted by revenue. Remember that there's a different cost for guests and members! 
SELECT facid, SUM( slots ) 
FROM Bookings
GROUP BY facid
HAVING SUM( slots ) >1000
ORDER BY facid